In [3]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

import time
import pickle

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# with open('artNouns.dat','rb') as file: # load
#     nouns = pickle.load(file)

In [4]:
with open('artAd_nouns.dat','rb') as file: # load
    ad_nouns = pickle.load(file)

전처리해주자 

TFIDF 기준으로 0.4 미만 단어는 불용어처리

In [10]:
with open('TFIDF.dic','rb') as file:
    TFIDF = pickle.load(file)

In [24]:
ad_nouns_refined = []
for i, doc in enumerate(ad_nouns):
    tmp = []
    for token in doc:
        if (TFIDF[i][token] >0.4):
            tmp.append(token)
    ad_nouns_refined.append(tmp)

In [26]:
dic = corpora.Dictionary(ad_nouns_refined)

In [27]:
type(dic)

gensim.corpora.dictionary.Dictionary

In [28]:
corpus = [dic.doc2bow(doc) for doc in ad_nouns_refined] #출현 횟수 vector 구성

In [29]:
s = time.time()
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus, 
                        num_topics=10, 
                        id2word = dic, 
                        passes=10
                        ) #LDA model 생성
print(time.time()-s)

260.83903193473816


In [30]:
ldamodel.save('lda.model')

In [31]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda.model')

In [32]:
ldamodel.show_topics(num_words=30) #토픽을 구성하는 주요단어들

[(0,
  '0.034*"여성" + 0.033*"인물" + 0.028*"얼굴" + 0.021*"그녀" + 0.016*"몸" + 0.011*"소녀" + 0.011*"여인" + 0.009*"제주" + 0.009*"자연" + 0.009*"표정" + 0.008*"옷" + 0.008*"남성" + 0.008*"그림" + 0.008*"여자" + 0.007*"관계" + 0.007*"초상" + 0.007*"그린" + 0.006*"회화" + 0.005*"소재" + 0.005*"섬" + 0.005*"남자" + 0.005*"초상화" + 0.005*"인체" + 0.004*"화가" + 0.004*"순간" + 0.004*"신체" + 0.004*"모델" + 0.004*"자화상" + 0.004*"손" + 0.003*"배경"'),
 (1,
  '0.006*"욕망" + 0.004*"자기" + 0.003*"내면" + 0.003*"자아" + 0.003*"의식" + 0.003*"감정" + 0.003*"수도" + 0.003*"이런" + 0.003*"상태" + 0.003*"죽음" + 0.003*"현대인" + 0.002*"불안" + 0.002*"무의식" + 0.002*"그런" + 0.002*"동물" + 0.002*"소통" + 0.002*"그렇" + 0.002*"결국" + 0.002*"문제" + 0.002*"그렇게" + 0.002*"타인" + 0.002*"그래서" + 0.002*"꿈" + 0.002*"누구" + 0.002*"심리" + 0.002*"본인" + 0.002*"가치" + 0.002*"나를" + 0.002*"않" + 0.002*"주체"'),
 (2,
  '0.014*"전통" + 0.007*"화가" + 0.007*"년대" + 0.007*"한국" + 0.005*"정신" + 0.004*"도자기" + 0.004*"역사" + 0.004*"기법" + 0.004*"동양" + 0.004*"중국" + 0.004*"산수" + 0.003*"생활" + 0.003*"민화" + 0.003*"미" + 0.003*"공예" +

#### 참고
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling

In [33]:
# Compute Perplexity
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.989126248865913


In [35]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=ad_nouns_refined, dictionary=dic, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.467156822233572


In [36]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

C:\Users\Chankoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.005290 -0.160926       1        1  18.374201
5     -0.107941 -0.104383       2        1  16.616310
8     -0.160616 -0.083725       3        1  15.735920
4      0.075033 -0.089562       4        1  13.436821
3     -0.172431  0.227697       5        1  10.080857
9      0.035112 -0.119229       6        1   7.123441
2     -0.098759  0.183850       7        1   6.757652
7      0.138362  0.077731       8        1   5.060625
6     -0.006967 -0.036076       9        1   4.785202
0      0.303496  0.104623      10        1   2.028966, topic_info=      Category          Freq  Term         Total  loglift  logprob
term                                                              
744    Default  10795.000000    사진  10795.000000  30.0000  30.0000
132    Default  10436.000000    도시  10436.000000  29.0000  29.0000
245    Default   6253.000000    인물   6253.000000  28.0000  28.0000
1370   Default   6429.000000    그녀   6429.000000  27.0000  27.0000
2133   Default   4361.000000    얼굴   4361.000000  26.0000  26.0000
874    Default   9094.000000     꽃   9094.000000  25.0000  25.0000
1561   Default   3652.000000    여성   3652.000000  24.0000  24.0000
2346   Default   4604.000000    기억   4604.000000  23.0000  23.0000
1445   Default   5554.000000     몸   5554.000000  22.0000  22.0000
949    Default   5247.000000    전통   5247.000000  21.0000  21.0000
30     Default   5802.000000   미술관   5802.000000  20.0000  20.0000
48     Default   7040.000000    역사   7040.000000  19.0000  19.0000
951    Default   8392.000000    조각   8392.000000  18.0000  18.0000
52     Default   4366.000000     월   4366.000000  17.0000  17.0000
1179   Default   5516.000000    장소   5516.000000  16.0000  16.0000
1746   Default   4735.000000    아트   4735.000000  15.0000  15.0000
544    Default   5161.000000     집   5161.000000  14.0000  14.0000
390    Default   3479.000000    아이   3479.000000  13.0000  13.0000
1840   Default   5251.000000    화가   5251.000000  12.0000  12.0000
349    Default   7321.000000    나무   7321.000000  11.0000  11.0000
1790   Default   7348.000000     빛   7348.000000  10.0000  10.0000
6260   Default   2663.000000    풍경   2663.000000   9.0000   9.0000
41     Default   4077.000000    서울   4077.000000   8.0000   8.0000
1278   Default   5906.000000    색채   5906.000000   7.0000   7.0000
741    Default   3557.000000    사물   3557.000000   6.0000   6.0000
14     Default   4137.000000    기획   4137.000000   5.0000   5.0000
789    Default   4721.000000    입니   4721.000000   4.0000   4.0000
496    Default   4040.000000     색   4040.000000   3.0000   3.0000
3      Default   3447.000000    개최   3447.000000   2.0000   2.0000
939    Default   6463.000000    욕망   6463.000000   1.0000   1.0000
...        ...           ...   ...           ...      ...      ...
1561   Topic10   3535.589111    여성   3652.627197   3.8651  -3.3844
4707   Topic10    252.752609   여성성    260.737762   3.8665  -6.0226
8447   Topic10    838.138977    남성    881.532837   3.8472  -4.8238
5080   Topic10    563.966919    소재    597.011597   3.8407  -5.2200
26412  Topic10     76.923180   권진규     79.832870   3.8605  -7.2122
8285   Topic10    312.218323    누드    353.177856   3.7744  -5.8113
5507   Topic10    272.117920   제주도    317.143951   3.7445  -5.9488
2795   Topic10   1176.630371    소녀   1569.649292   3.6094  -4.4846
2133   Topic10   2965.144775    얼굴   4361.907227   3.5117  -3.5603
2831   Topic10   1155.405518    여인   1573.949097   3.5885  -4.5028
4418   Topic10    504.568726   초상화    683.086304   3.5947  -5.3313
245    Topic10   3391.795654    인물   6253.505859   3.2859  -3.4259
4393   Topic10    782.964966    여자   1245.514160   3.4334  -4.8919
4417   Topic10    687.805359    초상   1136.710571   3.3953  -5.0215
4881   Topic10    880.241943     옷   1556.909424   3.3274  -4.7748
1370   Topic10   2203.938721    그녀   6429.916992   2.8269  -3.8570
2685   Topic10    527.6962

![1](https://user-images.githubusercontent.com/38183218/44544400-0ef41e00-a74d-11e8-894d-0260f2672cc9.PNG)

__How to find the optimal number of topics for LDA?__

My approach to finding the optimal number of topics is to build many LDA models with different values of number of topics (k) and pick the one that gives the highest coherence value.

Choosing a ‘k’ that marks the end of a rapid growth of topic coherence usually offers meaningful and interpretable topics. Picking an even higher value can sometimes provide more granular sub-topics.

If you see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.

The compute_coherence_values() (see below) trains multiple LDA models and provides the models and their corresponding coherence scores.

In [25]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dic, corpus=corpus, texts=niuns, start=2, limit=5, step=1)

In [ ]:
# Show graph
limit=5; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()